In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import astropy.io.fits as pyfits
import astropy.utils as autils
import requests
import json
import datetime
from pprint import pprint as pp
import os
import astropy.io.fits as fits
import astropy.io.ascii as ascii
import copy

# Jielai added modules f
import subprocess
import seaborn as sns

# Time Counter function
import time
def tic():
    tic.start = time.perf_counter()
def toc():
    elapsed_seconds = time.perf_counter() - tic.start
    return elapsed_seconds # fractional

# NOAO server Settings
natroot = 'https://astroarchive.noirlab.edu'
assert natroot == 'https://astroarchive.noirlab.edu', 'Notebook does NOT point to PRODUCTION'
print(f"Using server on {natroot}")
adsurl = f'{natroot}/api/adv_search'
print(f"adsurl = {adsurl}")
apiurl = f'{adsurl}/fasearch/?limit=200000'
print(f'Using API url: {apiurl}')

# Start the timer
print(f'Started on: {str(datetime.datetime.now())}')
tic() # Start timing the run of this notebook

Using server on https://astroarchive.noirlab.edu
adsurl = https://astroarchive.noirlab.edu/api/adv_search
Using API url: https://astroarchive.noirlab.edu/api/adv_search/fasearch/?limit=200000
Started on: 2022-02-13 22:07:16.327103


In [2]:
def  get_radec_maxmin(RAcentre,DECcentre,search_radius_deg, debug=False):

    dec_min = DECcentre - search_radius_deg
    dec_max = DECcentre + search_radius_deg
    if dec_min<-90.0: dec_min=-90.0
    if dec_max>90.0: dec_max=90.0
    if dec_min==-90.0 or dec_max==90.0:
        ra_min = 0
        ra_max = 360.0
    else:
        costerm = min(np.cos(dec_min*np.pi/180.0),np.cos(dec_max*np.pi/180.0))
        ra_min = RAcentre-search_radius_deg*1./costerm
        ra_max = RAcentre+search_radius_deg*1./costerm
        if ra_min<0: ra_min+=360.0
        if ra_max>360.0: ra_max-=360.0

    if debug:
        print('**** DEBUG: ',dec_min, dec_max)
        print('**** DEBUG: ',ra_min, ra_max)

    return ra_min,ra_max,dec_min,dec_max
def makedirs_(out_path):
    out_dir = '/'.join(out_path.split('/')[0:-1])
    os.makedirs(out_dir, exist_ok=True)
    return None

In [29]:
pw = 'XXX'
usrname = 'XXX'
search_radius_deg = 0.05
caldat1g = '1000-01-01'
caldat2g = '2022-02-10'
raw_dir = '/Users/jielaizhang/Desktop/' # where downloaded stuff is saved

In [25]:
f_pointings = '/Users/jielaizhang/src/KNTraP/notes/KNTraP.pointings.txt'
d_pointings                = ascii.read(f_pointings)
df_pointings               = pd.DataFrame(d_pointings.as_array())

# G-band

In [26]:
no_prior_gband_imaging_fileds = []

for pointing_name in df_pointings['field']:
    
    # get RA and DEC ranges
    df_pointings_selectedfield = df_pointings[df_pointings['field']==pointing_name]
    RAcentre          = df_pointings_selectedfield.iloc[0]['RA'] 
    DECcentre         = df_pointings_selectedfield.iloc[0]['Dec'] 
    ra_min,ra_max,dec_min,dec_max = get_radec_maxmin(RAcentre,DECcentre,search_radius_deg,debug=False)
    if ra_min>ra_max:
        raise RuntimeError('This needs to be fixed!!')
        # Need to perform two searches to account for overlap

    # Json needed to make a request to NOAO archive. 
    jj = {    
            "outfields" : [
                "md5sum","release_date","proposal","archive_filename","original_filename","url",
                "proc_type","prod_type","ifilter","exposure",
                "ra_center","dec_center","dateobs_min","dateobs_max","caldat","filesize",
                "seeing"
                ],
            "search" : [ ["instrument", 'decam'],["telescope", 'ct4m'],["obs_type", 'object'],
                         ['ra_center',ra_min,ra_max],['dec_center',dec_min,dec_max],
                         ["caldat","1000-01-01",caldat2g],
                         ["proc_type","instcal"],['prod_type','image'],
                         ['ifilter','g DECam SDSS c0001 4720.0 1520.0'] 
                       ]
        }
    ads_df = pd.DataFrame(requests.post(apiurl,json=jj).json()[1:])
    print('-----',pointing_name)
    print('Retrieved: ',len(ads_df))
    if len(ads_df) != 0:
        print(np.unique(ads_df['caldat']))
        print(np.unique(ads_df['exposure']))
    else:
        no_prior_gband_imaging_fileds.append(pointing_name)

----- S82sub8
Retrieved:  92
['2017-03-02' '2021-06-04' '2021-06-05' '2021-06-06' '2021-06-07'
 '2021-06-08' '2021-06-09']
[ 20. 104.]
----- 353A
Retrieved:  24
['2021-10-16' '2021-10-17' '2021-10-31' '2021-11-03' '2021-11-06'
 '2021-11-09' '2021-11-13' '2021-11-15' '2021-11-18' '2021-11-27'
 '2021-11-30' '2021-12-03' '2021-12-06' '2021-12-21' '2021-12-22'
 '2021-12-23' '2021-12-27' '2021-12-30' '2022-01-02' '2022-01-05'
 '2022-01-08' '2022-01-15' '2022-01-20' '2022-01-21']
[50.]
----- 353B
Retrieved:  24
['2021-10-16' '2021-10-17' '2021-10-31' '2021-11-03' '2021-11-06'
 '2021-11-09' '2021-11-13' '2021-11-15' '2021-11-18' '2021-11-27'
 '2021-11-30' '2021-12-03' '2021-12-06' '2021-12-21' '2021-12-22'
 '2021-12-23' '2021-12-27' '2021-12-30' '2022-01-02' '2022-01-05'
 '2022-01-08' '2022-01-15' '2022-01-20' '2022-01-21']
[50.]
----- 353C
Retrieved:  25
['2021-10-16' '2021-10-17' '2021-10-31' '2021-11-03' '2021-11-06'
 '2021-11-09' '2021-11-13' '2021-11-15' '2021-11-18' '2021-11-21'
 '2021-

# I-band

In [31]:
no_prior_iband_imaging_fileds = []

for pointing_name in df_pointings['field']:
    
    # get RA and DEC ranges
    df_pointings_selectedfield = df_pointings[df_pointings['field']==pointing_name]
    RAcentre          = df_pointings_selectedfield.iloc[0]['RA'] 
    DECcentre         = df_pointings_selectedfield.iloc[0]['Dec'] 
    ra_min,ra_max,dec_min,dec_max = get_radec_maxmin(RAcentre,DECcentre,search_radius_deg,debug=False)
    if ra_min>ra_max:
        raise RuntimeError('This needs to be fixed!!')
        # Need to perform two searches to account for overlap

    # Json needed to make a request to NOAO archive. 
    jj = {    
            "outfields" : [
                "md5sum","release_date","proposal","archive_filename","original_filename","url",
                "proc_type","prod_type","ifilter","exposure",
                "ra_center","dec_center","dateobs_min","dateobs_max","caldat","filesize",
                "seeing"
                ],
            "search" : [ ["instrument", 'decam'],["telescope", 'ct4m'],["obs_type", 'object'],
                         ['ra_center',ra_min,ra_max],['dec_center',dec_min,dec_max],
                         ["caldat","1000-01-01",caldat2g],
                         ["proc_type","instcal"],['prod_type','image'],
                         ['ifilter','i DECam SDSS c0003 7835.0 1470.0'] 
                       ]
        }
    ads_df = pd.DataFrame(requests.post(apiurl,json=jj).json()[1:])
    print('-----',pointing_name)
    print('Retrieved: ',len(ads_df))
    if len(ads_df) != 0:
        print(np.unique(ads_df['caldat']))
        print(np.unique(ads_df['exposure']))
    else:
        no_prior_iband_imaging_fileds.append(pointing_name)

----- S82sub8
Retrieved:  20
['2018-04-11' '2021-06-04' '2021-06-05' '2021-06-06' '2021-06-07'
 '2021-06-08' '2021-06-09']
[ 90. 150.]
----- 353A
Retrieved:  28
['2021-09-16' '2021-09-19' '2021-09-20' '2021-09-22' '2021-09-25'
 '2021-10-19' '2021-10-20' '2021-10-23' '2021-10-24' '2021-10-28'
 '2021-11-03' '2021-11-06' '2021-11-27' '2021-11-30' '2021-12-06'
 '2021-12-09' '2021-12-12' '2021-12-15' '2021-12-18' '2021-12-21'
 '2021-12-22' '2021-12-27' '2022-01-02' '2022-01-08' '2022-01-12'
 '2022-01-15' '2022-01-17' '2022-01-20']
[50.]
----- 353B
Retrieved:  28
['2021-09-16' '2021-09-19' '2021-09-20' '2021-09-22' '2021-09-25'
 '2021-10-19' '2021-10-20' '2021-10-23' '2021-10-24' '2021-10-28'
 '2021-11-03' '2021-11-06' '2021-11-27' '2021-11-30' '2021-12-06'
 '2021-12-09' '2021-12-12' '2021-12-15' '2021-12-18' '2021-12-21'
 '2021-12-22' '2021-12-27' '2022-01-02' '2022-01-08' '2022-01-12'
 '2022-01-15' '2022-01-17' '2022-01-20']
[50.]
----- 353C
Retrieved:  28
['2021-09-16' '2021-09-19' '2021-

----- KNTRAP6
Retrieved:  0
----- KNTRAP7
Retrieved:  0
----- KNTRAP8
Retrieved:  0
----- KNTRAP9
Retrieved:  0
----- KNTRAP10
Retrieved:  20
['2014-04-02' '2014-04-03']
[100.]
----- KNTRAP11
Retrieved:  0
----- KNTRAP12
Retrieved:  0
----- KNTRAP13
Retrieved:  0
----- KNTRAP14
Retrieved:  0
